# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os

api_key = os.getenv("GOOGLE_API_KEY")

## LLMChain

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [11]:
chat_model = ChatGoogleGenerativeAI(
    google_api_key=api_key,
    temperature=0.9,
    model="gemini-1.5-flash"
)

In [12]:
prompt_template = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [13]:
chain = LLMChain(llm=chat_model, prompt=prompt_template)

In [14]:
product = "Queen Size Sheet Set"

chain.run(product)

'The best name will depend on your brand\'s desired image and target audience. Here are some ideas, categorized by approach:\n\n**Elegant & Sophisticated:**\n\n* Royal Rest\n* Queen\'s Comfort\n* Serene Sheets\n* The Linen Loft\n* Slumber Supreme\n* Luxurious Linens\n\n**Simple & Direct:**\n\n* Queen Sheet Co.\n* Queen\'s Sheets\n* The Queen\'s Bed\n* Simply Queen\n* Perfect Slumber\n\n**Modern & Trendy:**\n\n* Cozy Kingdom\n* Dream Weaver Sheets\n* Sleep Sanctuary\n* The Sheet Society\n* Linen & Lace\n\n**Focusing on Quality/Material:**\n\n* (Material) Queen Sheets (e.g., Egyptian Cotton Queen Sheets)\n* Premium Queen Linens\n* The Finest Sheets\n\n**Playful & Unique:**\n\n* Queen Bee Sheets (if targeting a younger audience)\n* Night Owl Linens\n* Sleepyhead Sheets\n\n\n**Before choosing, consider:**\n\n* **Availability:** Check if the name is available as a website domain and social media handle.\n* **Target audience:**  Who are you trying to reach? A younger audience might respond b

## SimpleSequentialChain

In [15]:
from langchain.chains import SimpleSequentialChain

In [16]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=chat_model, prompt=first_prompt)

In [17]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=chat_model, prompt=second_prompt)

In [18]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [19]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
The best name will depend on the company's branding and target audience. Here are some options, categorized by approach:

**Elegant & Luxurious:**

* Royal Rest
* Empress Linens
* Sovereign Sheets
* Regal Sleep
* Velvet Dreams (if the sheets are particularly soft)
* The Queen's Chamber

**Simple & Straightforward:**

* Queen Size Sheets
* Queen's Rest
* The Sheet Set Company
* Cozy Queen
* Perfect Slumber

**Modern & Catchy:**

* Slumber & Co.
* Dream Weaver Linens
* Sheet Dreams
* Restful Nights
* The Linen Loft

**Focusing on a specific feature (if applicable):**

* (e.g., if they're Egyptian cotton)  Nile Nights
* (e.g., if they're sustainably sourced)  EcoSlumber
* (e.g., if they're exceptionally soft) Cloud Nine Linens


**Things to consider when choosing:**

* **Availability:** Check for trademark conflicts and domain name availability.
* **Memorability:**  Is the name easy to remember and pronounce?
* **Target audience:** Does the 

'Luxury bedding for discerning sleepers.  Unparalleled comfort and exquisite craftsmanship.\n'

## SequentialChain

In [20]:
from langchain.chains import SequentialChain

In [27]:
import pandas as pd

df = pd.read_csv("datasets/sample_data.csv")

df.head()

,Text,Score
0,Habiendo probado un par de otras marcas de gal...,5
1,Meine Katze liebt diese Leckerlis. Wenn ich si...,5
2,Un po' meno di quanto mi aspettassi. Tende ad ...,3
3,"First there was Frosted Mini-Wheats, in origin...",2
4,En ik wil de grafisch ontwerper feliciteren vo...,5


### Prompt template 1: Translate to English

In [67]:
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to English:"
    "\n\n{Review}"
)

# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=chat_model, prompt=first_prompt, 
                     output_key="English_Review"
                    )

### Prompt template 2: Summarize the review

In [68]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=chat_model, prompt=second_prompt, 
                     output_key="summary"
                    )

### Prompt template 3: Detect language

In [69]:
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)

# chain 3: input= Review and output= language
chain_three = LLMChain(llm=chat_model, prompt=third_prompt,
                       output_key="language"
                      )

### Prompt template 4: Follow up message

In [70]:
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=chat_model, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

### Overall Chain

In [71]:
# overall_chain: input= Review 
# and output= English_Review, summary, language and followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "language", "followup_message"],
    verbose=True
)

In [72]:
# first review in the dataset
review = df['Text'][0]

review

"Habiendo probado un par de otras marcas de galletas sándwich sin gluten, estas son las mejores del grupo.Son crujientes y fieles a la textura de las otras galletas 'reales' que no son sin gluten.Algunos podrían pensar que el relleno las hace un poco demasiado dulces,\\ \npero para mí eso solo significa que he satisfecho mi gusto por lo dulce más rápido.La versión de chocolate de Glutino es igual de buena y tiene un verdadero sabor a 'chocolate',\\ \nalgo que no está presente en las otras marcas sin gluten disponibles.\n"

In [79]:
response = overall_chain.invoke(review)

response



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Habiendo probado un par de otras marcas de galletas sándwich sin gluten, estas son las mejores del grupo.Son crujientes y fieles a la textura de las otras galletas 'reales' que no son sin gluten.Algunos podrían pensar que el relleno las hace un poco demasiado dulces,\\ \npero para mí eso solo significa que he satisfecho mi gusto por lo dulce más rápido.La versión de chocolate de Glutino es igual de buena y tiene un verdadero sabor a 'chocolate',\\ \nalgo que no está presente en las otras marcas sin gluten disponibles.\n",
 'English_Review': 'Having tried a couple of other brands of gluten-free sandwich cookies, these are the best of the bunch. They\'re crunchy and true to the texture of "real" non-gluten-free cookies. Some might find the filling a little too sweet, but for me, that just means I satisfied my sweet tooth faster.  Glutino\'s chocolate version is equally good and has a real chocolate flavor, something lacking in other available gluten-free brands.\n',
 'summary

In [80]:
response['Review']

"Habiendo probado un par de otras marcas de galletas sándwich sin gluten, estas son las mejores del grupo.Son crujientes y fieles a la textura de las otras galletas 'reales' que no son sin gluten.Algunos podrían pensar que el relleno las hace un poco demasiado dulces,\\ \npero para mí eso solo significa que he satisfecho mi gusto por lo dulce más rápido.La versión de chocolate de Glutino es igual de buena y tiene un verdadero sabor a 'chocolate',\\ \nalgo que no está presente en las otras marcas sin gluten disponibles.\n"

In [81]:
response['English_Review']

'Having tried a couple of other brands of gluten-free sandwich cookies, these are the best of the bunch. They\'re crunchy and true to the texture of "real" non-gluten-free cookies. Some might find the filling a little too sweet, but for me, that just means I satisfied my sweet tooth faster.  Glutino\'s chocolate version is equally good and has a real chocolate flavor, something lacking in other available gluten-free brands.\n'

In [82]:
response['summary']

'These gluten-free sandwich cookies have a superior crunchy texture to other brands, though some may find the filling overly sweet.\n'

In [83]:
response['language']

"That's Spanish.\n"

In [84]:
response['followup_message']

'Sí, estoy de acuerdo.  Las galletas son excepcionalmente crujientes,  una textura que realmente destaca entre las opciones sin gluten. Sin embargo,  la dulzura del relleno podría ser un inconveniente para algunos paladares,  quizás un poco excesiva.  En general, una buena opción para quienes buscan una galleta crujiente sin gluten, pero recomiendo probarlas antes de comprar una cantidad grande.\n'

`Note:` Followup message is in Spanish, because we specified language as spanish in prompt template 4

## Router Chain

In [85]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [86]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [87]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [88]:
destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=chat_model, prompt=prompt)

    # add it to the destination dict
    destination_chains[name] = chain
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [89]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=chat_model, prompt=default_prompt)

In [90]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [94]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations = destinations_str
)

router_prompt = PromptTemplate(
    template = router_template,
    input_variables = ["input"],
    output_parser = RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(chat_model, router_prompt)

In [96]:
chain = MultiPromptChain(router_chain = router_chain, 
                         destination_chains = destination_chains, 
                         default_chain = default_chain,
                         verbose=True
                        )

In [104]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by an idealized object called a black body.  A black body absorbs all incident electromagnetic radiation regardless of frequency or angle of incidence.  Because it absorbs all radiation, it also emits radiation at all frequencies, with the intensity of that radiation at each frequency depending only on the black body's temperature.  This emitted radiation follows a specific distribution described by Planck's law.  The hotter the body, the more intense the radiation and the shorter the wavelength of peak emission.\n"

In [100]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The problem "What is 2 + 2?" is a simple addition problem.\n\n**Component Parts:**\n\n* The first number is 2.\n* The second number is 2.\n* The operation is addition (+).\n\n**Solution:**\n\nAdding the two numbers together: 2 + 2 = 4\n\n**Answer:**\n\nThe answer to the question "What is 2 + 2?" is $\\boxed{4}$.\n'

In [103]:
chain.run("Who's hitler?")



> Entering new MultiPromptChain chain...
History: {'input': 'Who was Hitler?'}
> Finished chain.


'Adolf Hitler (1889-1945) was the dictator of Germany from 1933 to 1945, and the central figure of the Nazi Party and the ideology of Nazism.  He was not simply a political leader; he was a transformative figure who fundamentally reshaped German society and instigated a global conflict with devastating consequences.\n\nUnderstanding Hitler requires examining him within multiple contexts:\n\n**His Personal Background:**  Born in Austria, Hitler\'s early life was marked by perceived failure and resentment.  His experiences of poverty, artistic aspirations that went unfulfilled, and feelings of national humiliation fueled his intense nationalism and antisemitism. While his personal life was largely devoid of close, genuine relationships, he cultivated a charismatic public persona, exploiting his skills as an orator to connect with a disillusioned populace.  It\'s crucial to avoid romanticizing his personal struggles, however, as they don\'t excuse his actions.\n\n**The Political Context:*